In [1]:
!pip install googledrivedownloader
!pip install keras

    100% |████████████████████████████████| 327kB 2.3MB/s 


In [2]:
from google_drive_downloader import GoogleDriveDownloader as gdd

ids =["1o4FNPEsiJyTAkexCzR1P85WgaODJZTMs","1yiP1nS5bCKHYto0Px3f_BOUbyJJVdKRO",
"1f3wDVtDq1oedLJ0Hidn2gduttRacAtoD", "1Svv2H8HgF-gyE3GgHJ4Kgcpb7_l88tBl",
"1aJQwc58seuB4tyIV3tZCFQBbSsI0CAVJ", "1Q77JWfZLXc_BmFoj0rQoIojvtloTkk2f",
"1bYFaYWfmmAgfj4hyEQXSsY_PrMt9YF0l", "1ZKhkmCHBFrrocGhmAG55Wgkd-ewZmd9l",
"1Z7FiOG8iTfUBnogZw8SQOPd1NDjQ79CO"]

file_names = ["bed.pkl","bird.pkl", "cat.pkl", "dog.pkl", "down.pkl", "eight.pkl", "five.pkl", "four.pkl", "go.pkl"]
for i in range(len(ids)):
  a = gdd.download_file_from_google_drive(file_id= ids[i],
                                      dest_path="./"+file_names[i],
                                      unzip=False)

In [3]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.utils import np_utils
from keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from keras import backend as K
from keras.models import load_model
from keras.callbacks import EarlyStopping, ModelCheckpoint
K.set_image_dim_ordering('th')
import pickle

Using TensorFlow backend.


In [0]:
"""words list represents the "pkl" file names
    which have the same word as .wav file.
    The pkl file is dictionary with key as the word spoken
    and the value as numpy array where element is a (20 * 100) vector
    representing the mfcc vectors of the .wav file. """

words = ["bed", "down", "dog", "eight", "four", "cat","go","bird","five"]
#words = ["bed", "down", "dog"]

In [5]:
data = pickle.load( open( words[0]+".pkl", "rb" ) )
for keys in data:
    X = data[keys]
    Y = np.array([[keys]] * data[keys].shape[0])
    #print X_1.shape
    
    
for i in range(1,len(words)):
    data = pickle.load( open( words[i]+".pkl", "rb" ) )
    for keys in data:
        Y = np.append(Y, [[keys]] * data[keys].shape[0], axis = 0)
        X = np.append(X, data[keys], axis = 0)
        #print X_1.shape
        
print X.shape
print Y.shape
print Y

(18735, 20, 100)
(18735, 1)
[['bed']
 ['bed']
 ['bed']
 ...
 ['five']
 ['five']
 ['five']]


In [6]:
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(Y)
#print integer_encoded
onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
onehot_encoded = onehot_encoder.fit_transform(integer_encoded)
Y = onehot_encoded

/usr/local/lib/python2.7/dist-packages/sklearn/preprocessing/label.py:111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [7]:
"""Convolution Layer in Keras
   accepts input as 3d - height, width, depth(RGB channels).
   Since we do not have depth in text, we can convert it to a vector
   with a depth = 1"""

X = X.reshape(X.shape[0], 1, 20, 100)
print X.shape
print Y.shape
num_classes = Y.shape[1]
print num_classes

(18735, 1, 20, 100)
(18735, 9)
9


In [0]:
model = Sequential()
model.add(Conv2D(50, (5, 5), input_shape=(1, 20, 100), activation='relu', padding="same"))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(30, (3, 3), activation='relu', padding="same"))
#model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(15, (3, 3), activation='relu', padding="same"))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))


model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(50, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))
# Compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [9]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 50, 20, 100)       1300      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 50, 10, 50)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 30, 10, 50)        13530     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 15, 10, 50)        4065      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 15, 5, 25)         0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 15, 5, 25)         0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 1875)              0         
__________

In [10]:
file_path="CNN_val_loss.hdf5"
checkpoint = ModelCheckpoint(file_path, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
early = EarlyStopping(monitor="val_loss", mode="min", patience=20)
callbacks_list = [checkpoint, early] #early
model.fit(X, Y, validation_split=0.1, callbacks=callbacks_list, epochs=1, batch_size= 32)

Train on 16861 samples, validate on 1874 samples
Epoch 1/1
16861/16861 [==============================] - 9s 534us/step - loss: 1.6297 - acc: 0.4475 - val_loss: 2.6635 - val_acc: 0.0464

Epoch 00001: val_loss improved from inf to 2.66350, saving model to CNN_val_loss.hdf5


In [0]:
model = load_model(file_path)

In [19]:
model.fit(X, Y, validation_split=0.1, callbacks=callbacks_list, epochs=20, batch_size= 32)

Train on 16861 samples, validate on 1874 samples
Epoch 1/20
16861/16861 [==============================] - 7s 443us/step - loss: 0.3195 - acc: 0.8927 - val_loss: 0.6146 - val_acc: 0.8303

Epoch 00001: val_loss did not improve
Epoch 2/20
16861/16861 [==============================] - 7s 430us/step - loss: 0.3119 - acc: 0.8951 - val_loss: 0.4118 - val_acc: 0.8789

Epoch 00002: val_loss improved from 0.47445 to 0.41183, saving model to CNN_val_loss.hdf5
Epoch 3/20
 9664/16861 [================>.............] - ETA: 2s - loss: 0.2901 - acc: 0.9036

16861/16861 [==============================] - 7s 424us/step - loss: 0.2893 - acc: 0.9045 - val_loss: 0.5142 - val_acc: 0.8623

Epoch 00003: val_loss did not improve
Epoch 4/20
16861/16861 [==============================] - 7s 427us/step - loss: 0.2917 - acc: 0.9050 - val_loss: 0.8128 - val_acc: 0.7647

Epoch 00004: val_loss did not improve
Epoch 5/20
16861/16861 [==============================] - 7s 427us/step - loss: 0.2831 - acc: 0.9062 - val_loss: 0.7712 - val_acc: 0.7791

Epoch 00005: val_loss did not improve
Epoch 6/20
 1056/16861 [>.............................] - ETA: 6s - loss: 0.2740 - acc: 0.8996

16861/16861 [==============================] - 7s 428us/step - loss: 0.2559 - acc: 0.9166 - val_loss: 0.4459 - val_acc: 0.8645

Epoch 00006: val_loss did not improve
Epoch 7/20
16861/16861 [==============================] - 7s 428us/step - loss: 0.2596 - acc: 0.9146 - val_loss: 0.6383 - val_acc: 0.8346

Epoch 00007: val_loss did not improve
Epoch 8/20
16861/16861 [==============================] - 7s 428us/step - loss: 0.2445 - acc: 0.9190 - val_loss: 0.6644 - val_acc: 0.8234

Epoch 00008: val_loss did not improve
Epoch 9/20
   32/16861 [..............................] - ETA: 10s - loss: 0.5436 - acc: 0.7188

16861/16861 [==============================] - 7s 428us/step - loss: 0.2378 - acc: 0.9206 - val_loss: 0.4479 - val_acc: 0.8773

Epoch 00009: val_loss did not improve
Epoch 10/20
16861/16861 [==============================] - 7s 418us/step - loss: 0.2327 - acc: 0.9219 - val_loss: 0.5244 - val_acc: 0.8554

Epoch 00010: val_loss did not improve
Epoch 11/20
16861/16861 [==============================] - 7s 423us/step - loss: 0.2313 - acc: 0.9221 - val_loss: 0.3821 - val_acc: 0.8954

Epoch 00011: val_loss improved from 0.41183 to 0.38213, saving model to CNN_val_loss.hdf5
Epoch 12/20
   32/16861 [..............................] - ETA: 7s - loss: 0.1360 - acc: 0.9688

16861/16861 [==============================] - 7s 416us/step - loss: 0.2263 - acc: 0.9254 - val_loss: 0.6212 - val_acc: 0.8415

Epoch 00012: val_loss did not improve
Epoch 13/20
16861/16861 [==============================] - 7s 420us/step - loss: 0.2184 - acc: 0.9294 - val_loss: 0.7354 - val_acc: 0.8170

Epoch 00013: val_loss did not improve
Epoch 14/20
16861/16861 [==============================] - 7s 430us/step - loss: 0.2122 - acc: 0.9303 - val_loss: 0.6828 - val_acc: 0.8212



Epoch 00014: val_loss did not improve
Epoch 15/20
16861/16861 [==============================] - 7s 439us/step - loss: 0.2002 - acc: 0.9341 - val_loss: 0.4241 - val_acc: 0.8965

Epoch 00015: val_loss did not improve
Epoch 16/20
16861/16861 [==============================] - 7s 430us/step - loss: 0.2127 - acc: 0.9288 - val_loss: 0.5237 - val_acc: 0.8687

Epoch 00016: val_loss did not improve
Epoch 17/20
10880/16861 [==================>...........] - ETA: 2s - loss: 0.2062 - acc: 0.9310

16861/16861 [==============================] - 8s 448us/step - loss: 0.2136 - acc: 0.9288 - val_loss: 0.4310 - val_acc: 0.8959

Epoch 00017: val_loss did not improve
Epoch 18/20
16861/16861 [==============================] - 8s 446us/step - loss: 0.2002 - acc: 0.9362 - val_loss: 0.5924 - val_acc: 0.8538

Epoch 00018: val_loss did not improve
Epoch 19/20
16861/16861 [==============================] - 8s 451us/step - loss: 0.1995 - acc: 0.9335 - val_loss: 0.3816 - val_acc: 0.9029

Epoch 00019: val_loss improved from 0.38213 to 0.38164, saving model to CNN_val_loss.hdf5
Epoch 20/20
  416/16861 [..............................] - ETA: 7s - loss: 0.2093 - acc: 0.9351

16861/16861 [==============================] - 8s 462us/step - loss: 0.1866 - acc: 0.9391 - val_loss: 0.4562 - val_acc: 0.8965

Epoch 00020: val_loss did not improve


In [0]:
model = load_model(file_path)

In [0]:
predicted = model.predict(X)

In [0]:
from sklearn import metrics

In [23]:
metrics.log_loss(Y, predicted)

0.09585370831917868